## EFD2 data Analysis

`1th Sept to 18th Sept, 2023`

`Defect Date: 4th sept and 11th Sept, 2023`

### Fresh code

In [24]:
import pandas as pd
import pyodbc
import plotly.subplots as sp
import plotly.graph_objs as go

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=na0vm00024.apac.bosch.com;"
                        "Database=DB_MFC2DB_SQL;"
                        "Trusted_Connection=yes;")

start_date = pd.to_datetime('2023-07-11')
end_date = pd.to_datetime('2023-07-16')

num_days = (end_date - start_date).days + 1
num_rows = (num_days - 1) // 5 + 1
num_cols = min(num_days, 5)

fig = sp.make_subplots(rows=num_rows, cols=num_cols,
                       subplot_titles=[f"{date.date()}" for date in pd.date_range(start=start_date, periods=num_days, freq='D')],
                       shared_yaxes=True)

for i, current_date in enumerate(pd.date_range(start=start_date, end=end_date, freq='D')):
    row_num = (i // num_cols) + 1
    col_num = (i % num_cols) + 1

    start_time = pd.to_datetime(current_date + pd.DateOffset(hours=6)).strftime('%Y-%m-%d %H:%M:%S')
    end_time = pd.to_datetime(current_date + pd.DateOffset(hours=30)).strftime('%Y-%m-%d %H:%M:%S')

    sql = f"SELECT * FROM [EFD3] WHERE [TimeStamp] >= '{start_time}' AND [TimeStamp] < '{end_time}' ORDER BY [TimeStamp] ASC"
    df = pd.read_sql_query(sql, cnxn, parse_dates=True)

    df = df[["PartCount", "HEnergy1", "TEnergy1", "TimeStamp", "QTemp", "TemTempRH", "TemTempLH", "HTempLH", "HTempRH"]]
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df['Date'] = df['TimeStamp'].dt.date
    df['Time'] = df['TimeStamp'].dt.time
    start_time = pd.to_datetime('00:00:00.840000').time()
    end_time = pd.to_datetime('23:59:59.893000').time()
    full_df = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]
    max_energy_per_partcount = full_df.groupby('PartCount')['HEnergy1'].max().reset_index()

    scatter_trace = go.Scatter(x=max_energy_per_partcount['PartCount'], y=max_energy_per_partcount['HEnergy1'],
                               mode='markers', name=f"{current_date.date()}")

    fig.add_trace(scatter_trace, row=row_num, col=col_num)

# Add a border around the entire subplot for specific dates
border_dates = [pd.to_datetime('2023-09-04').date(), pd.to_datetime('2023-09-11').date()]
for row_num in range(1, num_rows + 1):
    for col_num in range(1, num_cols + 1):
        index = (row_num - 1) * num_cols + col_num - 1
        if index < len(fig.layout.annotations):  # Check if the index is within the range
            current_date = pd.to_datetime(fig.layout.annotations[index].text).date()
            if current_date in border_dates:
                fig['layout']['annotations'][index]['bordercolor'] = 'black'
                fig['layout']['annotations'][index]['borderwidth'] = 2

fig.update_layout(
    title=dict(
        text="<b>Partcount vs Max_Hardening Energy</b>",
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=20, color='black', family='Arial'),
        pad=dict(b=10)
    ),
    showlegend=False
)
fig.write_html("EFD2_HEnergy_1-18Sep.html")
# fig.show()


In [25]:
import pandas as pd
import pyodbc
import plotly.subplots as sp
import plotly.graph_objs as go

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=na0vm00024.apac.bosch.com;"
                        "Database=DB_MFC2DB_SQL;"
                        "Trusted_Connection=yes;")

start_date = pd.to_datetime('2023-07-11')
end_date = pd.to_datetime('2023-07-16')

num_days = (end_date - start_date).days + 1
num_rows = (num_days - 1) // 5 + 1
num_cols = min(num_days, 5)

fig = sp.make_subplots(rows=num_rows, cols=num_cols,
                       subplot_titles=[f"{date.date()}" for date in pd.date_range(start=start_date, periods=num_days, freq='D')],
                       shared_yaxes=True)

for i, current_date in enumerate(pd.date_range(start=start_date, end=end_date, freq='D')):
    row_num = (i // num_cols) + 1
    col_num = (i % num_cols) + 1

    start_time = pd.to_datetime(current_date + pd.DateOffset(hours=6)).strftime('%Y-%m-%d %H:%M:%S')
    end_time = pd.to_datetime(current_date + pd.DateOffset(hours=30)).strftime('%Y-%m-%d %H:%M:%S')

    sql = f"SELECT * FROM [EFD3] WHERE [TimeStamp] >= '{start_time}' AND [TimeStamp] < '{end_time}' ORDER BY [TimeStamp] ASC"
    df = pd.read_sql_query(sql, cnxn, parse_dates=True)

    df = df[["PartCount", "HEnergy1", "TEnergy1", "TimeStamp", "QTemp", "TemTempRH", "TemTempLH", "HTempLH", "HTempRH"]]
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df['Date'] = df['TimeStamp'].dt.date
    df['Time'] = df['TimeStamp'].dt.time
    start_time = pd.to_datetime('00:00:00.840000').time()
    end_time = pd.to_datetime('23:59:59.893000').time()
    full_df = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]
    max_energy_per_partcount = full_df.groupby('PartCount')['HTempRH'].max().reset_index()

    scatter_trace = go.Scatter(x=max_energy_per_partcount['PartCount'], y=max_energy_per_partcount['HTempRH'],
                               mode='markers', name=f"{current_date.date()}")

    fig.add_trace(scatter_trace, row=row_num, col=col_num)

# Add a border around the entire subplot for specific dates
border_dates = [pd.to_datetime('2023-09-04').date(), pd.to_datetime('2023-09-11').date()]
for row_num in range(1, num_rows + 1):
    for col_num in range(1, num_cols + 1):
        index = (row_num - 1) * num_cols + col_num - 1
        if index < len(fig.layout.annotations):  # Check if the index is within the range
            current_date = pd.to_datetime(fig.layout.annotations[index].text).date()
            if current_date in border_dates:
                fig['layout']['annotations'][index]['bordercolor'] = 'black'
                fig['layout']['annotations'][index]['borderwidth'] = 2

fig.update_layout(
    title=dict(
        text="<b>Partcount vs Max_Hardening Temperature RH</b>",
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=20, color='black', family='Arial'),
        pad=dict(b=10)
    ),
    showlegend=False
)
fig.write_html("EFD2_HtempRH_1-18Sep.html")
# fig.show()


In [26]:
import pandas as pd
import pyodbc
import plotly.subplots as sp
import plotly.graph_objs as go

cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=na0vm00024.apac.bosch.com;"
                        "Database=DB_MFC2DB_SQL;"
                        "Trusted_Connection=yes;")

start_date = pd.to_datetime('2023-07-11')
end_date = pd.to_datetime('2023-07-16')

num_days = (end_date - start_date).days + 1
num_rows = (num_days - 1) // 5 + 1
num_cols = min(num_days, 5)

fig = sp.make_subplots(rows=num_rows, cols=num_cols,
                       subplot_titles=[f"{date.date()}" for date in pd.date_range(start=start_date, periods=num_days, freq='D')],
                       shared_yaxes=True)

for i, current_date in enumerate(pd.date_range(start=start_date, end=end_date, freq='D')):
    row_num = (i // num_cols) + 1
    col_num = (i % num_cols) + 1

    start_time = pd.to_datetime(current_date + pd.DateOffset(hours=6)).strftime('%Y-%m-%d %H:%M:%S')
    end_time = pd.to_datetime(current_date + pd.DateOffset(hours=30)).strftime('%Y-%m-%d %H:%M:%S')

    sql = f"SELECT * FROM [EFD3] WHERE [TimeStamp] >= '{start_time}' AND [TimeStamp] < '{end_time}' ORDER BY [TimeStamp] ASC"
    df = pd.read_sql_query(sql, cnxn, parse_dates=True)

    df = df[["PartCount", "HEnergy1", "TEnergy1", "TimeStamp", "QTemp", "TemTempRH", "TemTempLH", "HTempLH", "HTempRH"]]
    df['TimeStamp'] = pd.to_datetime(df['TimeStamp'])
    df['Date'] = df['TimeStamp'].dt.date
    df['Time'] = df['TimeStamp'].dt.time
    start_time = pd.to_datetime('00:00:00.840000').time()
    end_time = pd.to_datetime('23:59:59.893000').time()
    full_df = df[(df['Time'] >= start_time) & (df['Time'] <= end_time)]
    max_energy_per_partcount = full_df.groupby('PartCount')['HTempLH'].max().reset_index()

    scatter_trace = go.Scatter(x=max_energy_per_partcount['PartCount'], y=max_energy_per_partcount['HTempLH'],
                               mode='markers', name=f"{current_date.date()}")

    fig.add_trace(scatter_trace, row=row_num, col=col_num)

# Add a border around the entire subplot for specific dates
border_dates = [pd.to_datetime('2023-09-04').date(), pd.to_datetime('2023-09-11').date()]
for row_num in range(1, num_rows + 1):
    for col_num in range(1, num_cols + 1):
        index = (row_num - 1) * num_cols + col_num - 1
        if index < len(fig.layout.annotations):  # Check if the index is within the range
            current_date = pd.to_datetime(fig.layout.annotations[index].text).date()
            if current_date in border_dates:
                fig['layout']['annotations'][index]['bordercolor'] = 'black'
                fig['layout']['annotations'][index]['borderwidth'] = 2

fig.update_layout(
    title=dict(
        text="<b>Partcount vs Max_Hardening Temperature LH</b>",
        x=0.5,
        y=0.95,
        xanchor='center',
        yanchor='top',
        font=dict(size=20, color='black', family='Arial'),
        pad=dict(b=10)
    ),
    showlegend=False
)
fig.write_html("EFD2_HTempLH_1-18Sep.html")
# fig.show()


### Clubbing HTML

In [27]:
from bs4 import BeautifulSoup


with open("EFD2_HEnergy_1-18Sep.html", "r", encoding="utf-8") as file11:
    content1 = file11.read()

with open("EFD2_HTempLH_1-18Sep.html", "r", encoding="utf-8") as file21:
    content2 = file21.read()

with open("EFD2_HtempRH_1-18Sep.html", "r", encoding="utf-8") as file31:
    content3 = file31.read()

# with open("file41.html", "r", encoding="utf-8") as file41:
#     content4 = file41.read()
    
# with open("file51.html", "r", encoding="utf-8") as file51:
#     content5 = file51.read()

# with open("file61.html", "r", encoding="utf-8") as file61:
#     content6 = file61.read()
    
# with open("file71.html", "r", encoding="utf-8") as file71:
#     content7 = file71.read()
    

soup1 = BeautifulSoup(content1, 'html.parser')
soup2 = BeautifulSoup(content2, 'html.parser')
soup3 = BeautifulSoup(content3, 'html.parser')
# soup4 = BeautifulSoup(content4, 'html.parser')
# soup5 = BeautifulSoup(content5, 'html.parser')
# soup6 = BeautifulSoup(content6, 'html.parser')
# soup7 = BeautifulSoup(content7, 'html.parser')


combined_content = f"{soup1}{soup2}{soup3}"

# Write the combined content to a new HTML file
with open("EFD3_11-16July.html", "w", encoding="utf-8") as combined_file:
    combined_file.write(combined_content)
